## 10.1 GroupBy 機制

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.653049,0.760444
1,a,two,0.901173,0.491377
2,b,one,0.742818,-0.225516
3,b,two,0.728147,-0.733247
4,a,one,0.435930,-0.736326


In [5]:
# 以 key1 的標籤做分組，計算 data1 的分組平均值
grouped = df['data1'].groupby(df['key1'])
grouped.mean()

key1
a    0.663384
b    0.735482
Name: data1, dtype: float64

In [6]:
# key為多個陣列組成的 list
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.544489
      two     0.901173
b     one     0.742818
      two     0.728147
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,0.544489,0.901173
b,0.742818,0.728147


In [9]:
# key為 Series
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005    0.901173
            2006    0.742818
Ohio        2005    0.690598
            2006    0.435930
Name: data1, dtype: float64

In [11]:
# 輸入欄位名稱當作分組 key
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.663384,0.171832
b,0.735482,-0.479382


In [12]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.544489  0.012059
     two   0.901173  0.491377
b    one   0.742818 -0.225516
     two   0.728147 -0.733247

In [13]:
# 利用 groupby 計算各分組的大小
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 疊代分組